In [2]:
import pandas
import math
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm

FNAME = '../cps09mar.dta'
df0 = pandas.read_stata(FNAME)

df = df0[ (df0.race == 2) ]

/home/erichschulman/anaconda2/lib/python2.7/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [18]:
#part b - model set up

#dropped variables Hispanic, black, Americian Indian, Asian, Mixed Race from the table in 4.1
y = pandas.DataFrame()
y = df['lwage']
X = df.loc[:,('educ','exp','female')]
X['exp_sq'] = X['exp']**2/100
X['male*union'] = (1.0-df['female']).multiply(df['union'])
X['female*union'] = df['female'].multiply(df['union'])

#Set up marital status dummy variables/interaction effects
maritalstatus = pandas.get_dummies(df['marital']).loc[:,(1,2,3,4,5,6)]
maritalstatus['married'] = maritalstatus[1] +maritalstatus[2] + maritalstatus[3]
maritalstatus['former'] = maritalstatus[4] + maritalstatus[5] +maritalstatus[6]
X['married*male'] = (1.0-df['female']).multiply(maritalstatus['married'])
X['married*female'] = df['female'].multiply(maritalstatus['married'])
X['former*male'] =  (1.0-df['female']).multiply(maritalstatus['married'])
X['former*female'] = df['female'].multiply(maritalstatus['former'])

X = sm.add_constant(X)

In [19]:
#part b

#There are 4 restrictions. b7=0, b8=0, b9=0, and b10=0

model = sm.GLS(y,X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            GLS Regression Results                            
==============================================================================
Dep. Variable:                  lwage   R-squared:                       0.243
Model:                            GLS   Adj. R-squared:                  0.242
Method:                 Least Squares   F-statistic:                     182.7
Date:                Wed, 21 Mar 2018   Prob (F-statistic):          1.25e-301
Time:                        22:04:38   Log-Likelihood:                -4145.7
No. Observations:                5135   AIC:                             8311.
Df Residuals:                    5125   BIC:                             8377.
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const              0.8652      0.054     16.044      0.000       0.759       0.971
educ               0.1191      0.003     35.673      0.000       0.113       0.126
exp                0.0240      0.002     10.198      0.000       0.019       0.029
female            -0.1196      0.024     -4.953      0.000      -0.167      -0.072
exp_sq            -0.0339      0.005     -7.206      0.000      -0.043      -0.025
male*union         0.1449      0.075      1.920      0.055      -0.003       0.293
female*union      -0.0130      0.078     -0.166      0.868      -0.167       0.141
married*male       0.0603      0.011      5.308      0.000       0.038       0.083
married*female     0.0632      0.025      2.555      0.011       0.015       0.112
former*male        0.0603      0.011      5.308      0.000       0.038       0.083
former*female     -0.0105      0.028     -0.371      0.711      -0.066       0.045
==============================================================================
Omnibus:                     2542.558   Durbin-Watson:                   1.815
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            66866.453
Skew:                          -1.811   Prob(JB):                         0.00
Kurtosis:                      20.303   Cond. No.                     3.67e+17
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 3.3e-29. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [21]:
#Part c-d

R = [[0,0,0,0,0,0,0,1,0,0,0],[0,0,0,0,0,0,0,0,1,0,0],[0,0,0,0,0,0,0,0,0,1,0],[0,0,0,0,0,0,0,0,0,0,1]]
wtest = model.wald_test(R)
wtest.summary()

#The P-Value is essentially 0, we can reject the null Hypothesis

'<F test: F=array([[ 6.00907376]]), p=8.04377300306e-05, df_denom=5125, df_num=4>'